In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
# =======================
# Load and prepare data
# =======================
df = pd.read_csv("dataset/T1.csv", parse_dates=['Date/Time'])

# Now convert to datetime with dayfirst=True (since your format is day month year)
df['Date/Time'] = pd.to_datetime(df['Date/Time'], dayfirst=True)
df = df.sort_values('Date/Time')

# Calculate time_diff first
df['time_diff'] = df['Date/Time'].diff().dt.total_seconds().div(60)
df['time_diff'].fillna(0, inplace=True)  # Handle first row NaN

# Example feature set (customize as needed)
features = ['Wind Speed (m/s)', 'Wind Direction (°)', 'Theoretical_Power_Curve (KWh)', 'time_diff']
target = 'LV ActivePower (kW)'

X = df[features]
y = df[target]

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"Features: {features}")

X shape: (50530, 4)
y shape: (50530,)
Features: ['Wind Speed (m/s)', 'Wind Direction (°)', 'Theoretical_Power_Curve (KWh)', 'time_diff']


In [8]:
# =======================
# Define models
# =======================
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42, max_depth=5),
    "Random Forest": RandomForestRegressor(random_state=42, n_estimators=100)
}

In [9]:
# =======================
# Cross-validation setup
# =======================
tscv = TimeSeriesSplit(n_splits=5)

results = []

for name, model in models.items():
    mae_list, rmse_list, r2_list = [], [], []
    
    for train_idx, test_idx in tscv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        # Build pipeline with scaling for linear models
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', model)
        ])
        
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        mae_list.append(mae)
        rmse_list.append(rmse)
        r2_list.append(r2)
    
    results.append({
        "Model": name,
        "MAE": np.mean(mae_list),
        "RMSE": np.mean(rmse_list),
        "R²": np.mean(r2_list)
    })

In [10]:
# =======================
# Results summary
# =======================
results_df = pd.DataFrame(results)
print(results_df.sort_values("RMSE"))

               Model         MAE        RMSE        R²
0  Linear Regression  210.769411  317.006411  0.934226
1      Decision Tree  191.397264  408.963192  0.889846
2      Random Forest  201.225842  449.743006  0.867407
